In [ ]:
import pandas as pd
import os
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc


### Single year (2023) explorations

In [ ]:
df = pd.read_csv('datafiles/combined_df_2023.csv', index_col = 0)

In [ ]:
df.head()

In [ ]:
df.columns

### Sorting by ladder score

In [ ]:
ladder_sorted_df = df.sort_values(by=['Ladder score'], ascending=False)

In [ ]:
top_10_countries = ladder_sorted_df.head(10)
bottom_10_countries = ladder_sorted_df.tail(10)

In [ ]:
bottom_10_countries

### Grouping by regions and numeric cols definition

In [ ]:
grouped_regions = df.groupby(by=['Regional Indicator'], as_index=False)

In [ ]:
from pandas.api.types import is_numeric_dtype

In [ ]:
numeric_cols = []
non_numeric_cols = []

In [ ]:
for col in df.columns:
    if is_numeric_dtype(df[col]):
        numeric_cols.append(col)
    else:
        non_numeric_cols.append(col)    

In [ ]:
numeric_cols

In [ ]:
grouped_regions.dtypes

In [ ]:
grouped_regions_mean = grouped_regions[[*numeric_cols]].mean()

In [ ]:
grouped_regions_mean.sort_values(by="Ladder score", inplace=True, ascending=False)

In [ ]:
grouped_regions_mean

#### Ladder score and GDP for each country bar-line

In [ ]:
ladder_gdp_fig = px.bar(grouped_regions_mean, y='Ladder score', x='Regional Indicator')
ladder_gdp_fig.add_trace(go.Scatter(x=grouped_regions_mean['Regional Indicator'],
                                    y=grouped_regions_mean['Logged GDP per capita'],
                                    line=dict(color='#ff7f0e'),
                                    name='GDP',
                                    mode='lines'))

### ========== Dash app ==========

In [ ]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

#### graph column/options definitions

In [ ]:
df_columns = list(df.columns)
except_columns = ['Ladder score standard error',
       'Ladder score upper whisker', 'Ladder score lower whisker',
        'Regional Indicator']
# columns for metric graphs
metric_graph_columns = [col for col in df_columns if col not in except_columns]

#### Metric vs ladder score for every country scatter

In [ ]:
metric_vs_ladder_controls = dbc.Card(
    [
        html.Div(
            [
            dbc.Label("Metric"),
            dcc.Dropdown(
                id='metric-vs-ladder-selection',
                value = 'Social support',
                options = [col for col in metric_graph_columns if col != 'Ladder score']
            )
            ]),
    ],
    body=True,
)

In [ ]:
metric_vs_ladder_container = dbc.Container([
    html.H1(children = 'Chosen metric vs ladder score', style={'textAlign': 'center'}),
    html.P(id='metric-vs-ladder-test'),
    dbc.Row(
        [
            dbc.Col(metric_vs_ladder_controls, md=4),
            dbc.Col(dcc.Graph(id='metric-vs-ladder-graph'), md=8),
        ],
        align="center",
    ),
], fluid=True)

In [ ]:
@app.callback(
    [
        Output('metric-vs-ladder-graph', 'figure'),
        Output('metric-vs-ladder-test', 'children')
    ],
    [
        Input('metric-vs-ladder-selection', 'value'),
    ]
)
def update_metric_vs_ladder_graph(metric):
    fig = px.scatter(df, y='Ladder score', x=metric, hover_data='Country', color='Regional Indicator',)
    return fig, f'metric: {metric}'
    # return fig

#### Metric vs life expectancy for every country scatter

In [ ]:
metric_vs_life_controls = dbc.Card(
    [
        html.Div(
            [
            dbc.Label("Metric"),
            dcc.Dropdown(
                id='metric-vs-life-selection',
                value = 'Social support',
                options = [col for col in metric_graph_columns if col != 'Life expectancy']
            )
            ]),
    ],
    body=True,
)

In [ ]:
metric_vs_life_container = dbc.Container([
    html.H1(children = 'Chosen metric vs life expectancy', style={'textAlign': 'center'}),
    html.P(id='metric-vs-life-test'),
    dbc.Row(
        [
            dbc.Col(metric_vs_life_controls, md=4),
            dbc.Col(dcc.Graph(id='metric-vs-life-graph'), md=8),
        ],
        align="center",
    ),
], fluid=True)

In [ ]:
@app.callback(
    [
        Output('metric-vs-life-graph', 'figure')
        ,Output('metric-vs-life-test', 'children')
    ],
    [
        Input('metric-vs-life-selection', 'value'),
    ]
)
def update_metric_vs_life_graph(metric):
    fig = px.scatter(df, y='Life expectancy', x=metric, hover_data='Country', color='Regional Indicator')
    return fig, f'metric: {metric}'
    # return fig

In [ ]:
# px.scatter(df, y='Ladder score', x='Social support', hover_data='Country', color='Regional Indicator', trendline='ols')

#### Within region two metric choosable scatter for countries within chosen region

In [ ]:
region_xy_controls = dbc.Card(
    [
        html.Div(
            [
            dbc.Label("Regional Indicator"),
            dcc.Dropdown(
                id='region-region-selection',
                value = 'Western Europe',
                options = list(df['Regional Indicator'].unique())
            )
            ]),
        html.Div(
            [
            dbc.Label("Metric x"),
            dcc.Dropdown(
                id='region-x-selection',
                value = 'Social support',
                options = metric_graph_columns
            )
            ]),
        html.Div(
            [
            dbc.Label("Metric y"),
            dcc.Dropdown(
                id='region-y-selection',
                value = 'Freedom',
                options = metric_graph_columns
            )
            ]),
    ],
    body=True,
)

In [ ]:
region_xy_container = dbc.Container([
    html.H1(children = 'Within region two metric choosable scatter for countries within chosen region', style={'textAlign': 'center'}),
    html.P(id='region-xy-test'),
    dbc.Row(
        [
            dbc.Col(region_xy_controls, md=4),
            dbc.Col(dcc.Graph(id='region-xy-graph'), md=8),
        ],
        align="center",
    ),
], fluid=True)

In [ ]:
@app.callback(
    [
        Output('region-xy-graph', 'figure'),
        Output('region-xy-test', 'children'),
    ],
    [
        Input('region-region-selection', 'value'),
        Input('region-x-selection', 'value'),
        Input('region-y-selection', 'value'),
    ]
)
def update_region_xy_graph(region, metric_x, metric_y):
    region_df = df[df['Regional Indicator'] == region]
    fig = px.scatter(region_df, y=metric_y, x=metric_x, hover_data='Country', color='Regional Indicator',
                    trendline = 'ols')
    return fig, f'region df shape: {region_df.shape}'
    # return fig

### Static comparisons

##### dataframe fixes

In [ ]:
# df[df['Urban_population'].isna()]['Country']

In [ ]:
# df['Urban_population'] = df['Urban_population'].str.replace(',', '')

In [ ]:
# df['Urban_population'] = df['Urban_population'].astype(float) / 1000000

In [ ]:
# df['Gross primary education enrollment (%)'] = df['Gross primary education enrollment (%)'].str.replace('%', '').astype(float)

In [ ]:
# df['Gross tertiary education enrollment (%)'] = df['Gross tertiary education enrollment (%)'].str.replace('%', '').astype(float)

In [ ]:
# df['Forested Area (%)'] = df['Forested Area (%)'].str.replace('%', '').astype(float)

In [ ]:
# df['Co2-Emissions'] = df['Co2-Emissions'].str.replace('NaN', '0')

In [ ]:
# df['Co2-Emissions'] = pd.to_numeric(df['Co2-Emissions'], errors='coerce')

In [ ]:
df.to_csv('datafiles/combined_df_2023.csv')

##### figures

In [ ]:
physicians_vs_mortality_fig = px.scatter(df, x='Physicians per thousand', y='Infant mortality', color='Regional Indicator', trendline='ols')
urbanpop_vs_primary_enrollment_fig = px.scatter(df, x='Urban_population', y='Gross primary education enrollment (%)',
                                                color='Regional Indicator', trendline='ols')
urbanpop_vs_tertiary_enrollment_fig = px.scatter(df, x='Urban_population', y='Gross tertiary education enrollment (%)',
                                                color='Regional Indicator', trendline='ols')

forestation_vs_co2_fig = px.scatter(df, x='Forested Area (%)', y='Co2-Emissions', color='Regional Indicator', trendline='ols')
gasoline_vs_co2_fig = px.scatter(df, x='Gasoline Price', y='Co2-Emissions', color='Regional Indicator', trendline='ols')
armed_vs_co2_fig = px.scatter(df, x='Armed Forces size', y='Co2-Emissions', color='Regional Indicator', trendline='ols')
density_vs_co2_fig = px.scatter(df, x='Density (P/Km2)', y='Co2-Emissions', color='Regional Indicator', trendline='ols')

# density_vs_co2_fig = px.scatter(df, x='Density (P/Km2)', y='Co2-Emissions', color='Regional Indicator', trendline='ols')
gdp_vs_businessrank_fig = px.scatter(df, x='Logged GDP per capita', y='Starting a Business rank', color='Regional Indicator', trendline='ols')

In [ ]:
physicians_vs_mortality_fig_size = px.scatter(df, x='Physicians per thousand', y='Infant mortality', color='Regional Indicator', trendline='ols',
                                             size='Ladder score')
urbanpop_vs_primary_enrollment_fig_size = px.scatter(df, x='Urban_population', y='Gross primary education enrollment (%)',
                                                color='Regional Indicator', trendline='ols', size='Ladder score')
urbanpop_vs_tertiary_enrollment_fig_size = px.scatter(df, x='Urban_population', y='Gross tertiary education enrollment (%)',
                                                color='Regional Indicator', trendline='ols', size='Ladder score')

forestation_vs_co2_fig_size = px.scatter(df, x='Forested Area (%)', y='Co2-Emissions', color='Regional Indicator', trendline='ols',
                                        size='Ladder score')
gasoline_vs_co2_fig_size = px.scatter(df, x='Gasoline Price', y='Co2-Emissions', color='Regional Indicator', trendline='ols', size='Ladder score')
armed_vs_co2_fig_size = px.scatter(df, x='Armed Forces size', y='Co2-Emissions', color='Regional Indicator', trendline='ols', size='Ladder score')
density_vs_co2_fig_size = px.scatter(df, x='Density (P/Km2)', y='Co2-Emissions', color='Regional Indicator', trendline='ols', size='Ladder score')

gdp_vs_businessrank_fig_size = px.scatter(df, x='Logged GDP per capita', y='Starting a Business rank', color='Regional Indicator', trendline='ols', size='Ladder score')

In [ ]:
static_figures = [
    physicians_vs_mortality_fig,
    urbanpop_vs_primary_enrollment_fig,
    urbanpop_vs_tertiary_enrollment_fig,
    forestation_vs_co2_fig,
    gasoline_vs_co2_fig,
    armed_vs_co2_fig,
    density_vs_co2_fig,
    gdp_vs_businessrank_fig
]

In [ ]:
static_figures_size = [
    physicians_vs_mortality_fig_size,
    urbanpop_vs_primary_enrollment_fig_size,
    urbanpop_vs_tertiary_enrollment_fig_size,
    forestation_vs_co2_fig_size,
    gasoline_vs_co2_fig_size,
    armed_vs_co2_fig_size,
    density_vs_co2_fig_size,
    gdp_vs_businessrank_fig_size
]

##### static figures layout

In [ ]:
static_figures_rows = []

len_static = len(static_figures)

for fig_index in range(len_static//2):
    static_figures_row = dbc.Row(
        [
            dbc.Col(dcc.Graph(id=f'static-graph-{fig_index*2}', figure=static_figures[fig_index*2])),
            dbc.Col(dcc.Graph(id=f'static-graph-{fig_index*2 + 1}', figure=static_figures[fig_index*2 + 1])),
        ],
        align="center",
    )
    static_figures_rows.append(static_figures_row)

static_figures_container = dbc.Container([
    html.H1(children = 'Static figures', style={'textAlign': 'center'}),
    # html.P(id='region-xy-test'),
    *static_figures_rows,
], fluid=True)

##### corr matrix

In [ ]:
corr_except = ['Ladder score lower whisker', 'Ladder score upper whisker', 'Ladder score stanard error']
corr_columns = [col for col in numeric_cols if col not in corr_except]
df_corr = df[numeric_cols].corr()

corr_map = go.Heatmap(x=df_corr.columns, y=df_corr.columns, z=df_corr,
                     colorscale=px.colors.diverging.RdBu)

corr_title = 'Correlation Matrix'

corr_layout = go.Layout(
    title_text=corr_title,  
    width=900, 
    height=900,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    # yaxis_autorange='reversed'
)
corr_fig=go.Figure(data=[corr_map], layout=corr_layout)

In [ ]:
corr_container = dbc.Container([
    html.H1(children = 'Correlation matrix', style={'textAlign': 'center'}),
    dcc.Graph(id='corr-graph-content',
             figure = corr_fig)
], fluid=True)

##### histogram multi-figure

In [ ]:
# to_hist_cols = ['Infant']

### Histogram multi-figure

In [ ]:
metric_hist_controls = dbc.Card(
    [
        html.Div(
            [
            dbc.Label("Metric"),
            dcc.Dropdown(
                id='hist-metric-selection',
                value = 'Social support',
                options = [col for col in metric_graph_columns if col != 'Ladder score']
            )
            ]),
    ],
    body=True,
)

In [ ]:
metric_hist_container = dbc.Container([
    html.H1(children = 'Metric histogram', style={'textAlign': 'center'}),
    html.P(id='hist-metric-test'),
    dbc.Row(
        [
            dbc.Col(metric_hist_controls, md=4),
            dbc.Col(dcc.Graph(id='hist-metric-graph'), md=8),
        ],
        align="center",
    ),
], fluid=True)

In [ ]:
@app.callback(
    [
        Output('hist-metric-graph', 'figure'),
        Output('hist-metric-test', 'children')
    ],
    [
        Input('hist-metric-selection', 'value'),
    ]
)
def update_hist_metric_graph(metric):
    # fig = px.scatter(df, y='Ladder score', x=metric, hover_data='Country', color='Regional Indicator',)
    fig = px.histogram(df, x=metric, color='Regional Indicator')
    return fig, f'metric: {metric}'
    # return fig

### Spine graphs by spreads / relation to median

In [ ]:
df.describe()

In [ ]:
spine_columns = ['Social support', 'Life expectancy', 'Freedom', 'Urban_population']

In [ ]:
social_median = df['Social support'].median()
life_median = df['Life expectancy'].median()
freedom_median = df['Freedom'].median()
urban_median = df['Urban_population'].median()

In [ ]:
medians = [social_median, life_median, freedom_median, urban_median]

In [ ]:
median_columns = [f'{col} above median' for col in spine_columns]

In [ ]:
median_columns

In [ ]:
for col_index, col in enumerate(median_columns):
    df[col] = df.apply(
    lambda row: True if row[spine_columns[col_index]] > medians[col_index] else False,
    axis = 1
    )

In [ ]:
median_col_True_means = []
for col in median_columns:
    median_mean = df[df[col] == True]['Ladder score'].mean()
    median_col_True_means.append(median_mean)

In [ ]:
median_col_False_means = []
for col in median_columns:
    median_mean = df[df[col] == False]['Ladder score'].mean()
    median_col_False_means.append(median_mean)

In [ ]:
median_col_False_means = [col * -1 for col in median_col_False_means]

In [ ]:
# ladder_above_median_freedom = df[df['Freedom above median'] == True]['Ladder score'].mean()

In [ ]:
# ladder_above_median_freedom

In [ ]:
# px.bar(y=[ladder_above_median_freedom,], x=['Freedom',])

In [ ]:
ladder_vs_medians_fig = go.Figure(
    data = [
        go.Bar(name = 'above median',
               y = spine_columns,
               x = median_col_True_means,
               orientation='h',
                marker=dict(color='#DD5555',
                line=dict(
                color='rgba(0,0,0,1.0)', width=0.5)),
                hoverinfo='none'
              ),
        go.Bar(name = 'below median',
               y = spine_columns,
               x = median_col_False_means,
               orientation='h',
                marker=dict(color='#5555DD',
                line=dict(
                color='rgba(0,0,0,1.0)', width=0.5)),
                hoverinfo='none'
              ),
    ],
)
ladder_vs_medians_fig.update_layout(barmode='relative')

In [ ]:
median_col_False_means = [col * -1 for col in median_col_False_means]

In [ ]:
median_vs_ladder_vertical_fig = go.Figure(
    data = [
        go.Bar(name = 'above median', x = spine_columns, y = median_col_True_means),
        go.Bar(name = 'below median', x = spine_columns, y = median_col_False_means),
    ],
)
median_vs_ladder_vertical_fig.update_layout(barmode='group',
                                           yaxis_title="Ladder score",
                                           xaxis_title="Metric")

In [ ]:
median_vs_ladder_container = dbc.Container([
    html.H1(children = 'Spine graphs by spreads', style={'textAlign': 'center'}),
    dcc.Graph(id='median-vs-ladder-graph-content',
             figure = median_vs_ladder_vertical_fig)
], fluid=True)

### Radar plots 

In [ ]:
# px.line_polar(r=median_col_True_means, theta = spine_columns)
grouped_by_region = df.groupby(by='Regional Indicator')[numeric_cols]

In [ ]:
grouped_by_region_means = grouped_by_region.mean()

In [ ]:
grouped_by_region_means

In [ ]:
work_means = grouped_by_region_means[['Agriculture', 'Industry', 'Service']]

In [ ]:
work_means.loc['Central and Eastern Europe', :].values

In [ ]:
work_means_regions = list(work_means.index)[1:]

In [ ]:
radar_colors = ['#466365', '#B49A67', '#01BAEF', '#ED4D6E', '#BAA5FF', '#FFB627', '#98CE00', '#DD1C1A', '#CC7178']

In [ ]:
work_radar_fig = px.line_polar(
    theta = ['Agriculture', 'Industry', 'Service'], r = work_means.loc[list(work_means.index)[0], :].values,
    line_close=True
)

In [ ]:
for r_index, region in enumerate(work_means_regions):
    # work_radar_fig.add_trace(
    #     px.line_polar(
    #     theta = ['Agriculture', 'Industry', 'Service'], r = work_means.loc[region, :].values,
    #     line_close=True, mark).data[0]
    # )
    work_radar_fig.add_trace(
        go.Scatterpolar(
          r=work_means.loc[region, :].values,
          theta=['Agriculture', 'Industry', 'Service'],
          name=region
    )
    )

In [ ]:
work_means

In [ ]:
work_radar_fig

In [ ]:
# parallel_fig = px.parallel_coordinates(work_means, color = work_means.index, dimensions = work_means.columns,
#                                        color_continuous_scale=px.colors.diverging.Tealrose)

In [ ]:
parallel_fig = px.parallel_coordinates(work_means, dimensions = work_means.columns,
                                       color_continuous_scale=px.colors.diverging.Tealrose)

In [ ]:
parallel_fig.show()

### Percentage donuts / histograms

In [ ]:
# forest_cuts = pd.cut(x=df['Forested Area (%)'].values, bins=10)

In [ ]:
# forest_cuts.to_list()

In [ ]:
# cuts_fig = go.Figure(data=[go.Pie(values=forest_cuts, hole=.3)])

In [ ]:
px.histogram(df, x='Forested Area (%)', nbins=10, color='Regional Indicator')

In [ ]:
forest_vs_agriculture_hist_fig = go.Figure()

In [ ]:
forest_vs_agriculture_hist_fig.add_trace(
    # px.histogram(df, x='Forested Area (%)', nbins=10).data[0]
    go.Histogram(x=df['Forested Area (%)'], nbinsx = 10, name='Forested Area (%)')
)
forest_vs_agriculture_hist_fig.add_trace(
    # px.histogram(df, x='Agriculture', nbins=10).data[0]
    go.Histogram(x=df['Agriculture']*100, nbinsx = 10, name='Agriculture')
)

forest_vs_agriculture_hist_fig.update_layout(barmode='overlay')
forest_vs_agriculture_hist_fig.update_traces(opacity=0.75)
forest_vs_agriculture_hist_fig.show()

In [ ]:
forest_vs_industry_hist_fig = go.Figure()
forest_vs_industry_hist_fig.add_trace(
    # px.histogram(df, x='Forested Area (%)', nbins=10).data[0]
    go.Histogram(x=df['Forested Area (%)'], nbinsx = 10, name='Forested Area (%)')
)
forest_vs_industry_hist_fig.add_trace(
    # px.histogram(df, x='Agriculture', nbins=10).data[0]
    go.Histogram(x=df['Industry']*100, nbinsx = 10, name='Industry')
)

forest_vs_industry_hist_fig.update_layout(barmode='overlay')
forest_vs_industry_hist_fig.update_traces(opacity=0.75)
forest_vs_industry_hist_fig.show()

In [ ]:
hist_with_boxes = px.histogram(df, x='Ladder score', color='Regional Indicator',
             marginal='violin', height=800)
hist_with_boxes

### Latitude , longitude

In [ ]:
px.scatter(df, x='Latitude', y='Agriculture')

In [ ]:
px.scatter(df, x='Longitude', y='Agriculture', color='Regional Indicator')

In [ ]:
px.scatter(df, x='Longitude', y='Latitude', size=df['Agriculture'].fillna(0), color='Regional Indicator')

### Choropleth

In [ ]:
# get ISO codes
iso_df = pd.read_csv('datafiles/world_countries_with_sectors.csv')
iso_df_to_merge = iso_df[['Country', 'Code']]

In [ ]:
df_with_iso = df.merge(iso_df_to_merge, how='left', on='Country')

In [ ]:
ladder_choropleth = px.choropleth(df_with_iso, locations='Code', color='Ladder score',
             hover_name='Country', color_continuous_scale=px.colors.sequential.Plasma,
             width=800, height=600)

In [ ]:
life_choropleth = px.choropleth(df_with_iso, locations='Code', color='Life expectancy',
             hover_name='Country', color_continuous_scale=px.colors.sequential.Plasma,
             width=800, height=600)
life_choropleth.show()

In [ ]:
gdp_choropleth = px.choropleth(df_with_iso, locations='Code', color='Logged GDP per capita',
             hover_name='Country', color_continuous_scale=px.colors.sequential.Plasma,
             width=800, height=600)
gdp_choropleth.show()

In [ ]:
life_bubble_map = px.scatter_geo(df_with_iso, locations='Code', color='Regional Indicator',
                                hover_name='Country', size='Life expectancy',
                                projection='natural earth')
life_bubble_map.show()

### ternary?

In [ ]:
px.scatter_ternary(df, a='Agriculture', b='Service', c='Industry', hover_name='Country',
                   size='Ladder score', size_max=15, color='Regional Indicator')
                  # color_discrete_map = {}

### others

In [ ]:
df[['Logged GDP per capita', 'Life expectancy', 'Population']].dtypes

In [ ]:
df['Population'] = pd.to_numeric(df['Population'].str.replace(',', ''), errors='coerce')

In [ ]:
px.scatter(df, x='Logged GDP per capita', y='Life expectancy', size=df['Population'].fillna(0), color='Regional Indicator',
          hover_name='Country', size_max=15)

### App layout and run

In [ ]:
app.layout = dbc.Container([
    metric_vs_ladder_container,
    html.Hr(),
    metric_vs_life_container,
    html.Hr(),
    region_xy_container,
    html.Hr(),
    metric_hist_container,
    html.Hr(),
    static_figures_container,
    html.Hr(),
    corr_container,
    html.Hr(),
    median_vs_ladder_container,
    html.Hr(),
], fluid=True)

In [ ]:
app.run(debug=True)

### ========== tests ============

#### Trendline fig

In [ ]:
trend_fig = px.scatter(df, x='Infant mortality', y='Physicians per thousand', trendline = 'ols')
trend_fig.show()

In [ ]:
results = px.get_trendline_results(trend_fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

#### Overall trendline fig

In [ ]:
overall_fig = px.scatter(df, y='Life expectancy', x='Agriculture', hover_data='Country', color='Regional Indicator',
                        trendline='ols', trendline_scope='overall')
overall_fig.show()

#### size_fig

In [ ]:
size_fig = px.scatter(df, x="Logged GDP per capita", y="Life expectancy", size="Ladder score",
                 color="Regional Indicator", hover_data='Country')

In [ ]:
size_fig.show()

#### dataframe with more data

In [ ]:
world_df = pd.read_csv('datafiles/world-data-2023.csv')

In [ ]:
world_df.columns

In [ ]:
world_df

In [ ]:
px.scatter(world_df, x='Physicians per thousand', y='Infant mortality', trendline='ols')

In [ ]:
list(world_df['Country'])

In [ ]:
metric_graph_columns

In [ ]:
corr_except = ['Ladder score lower whisker', 'Ladder score upper whisker', 'Ladder score stanard error']

In [ ]:
corr_columns = [col for col in numeric_cols if col not in corr_except]

In [ ]:
df_corr = df[numeric_cols].corr()

In [ ]:
# corr_map = go.Heatmap(x=df_corr.columns, y=df_corr.columns, z = df_corr,
#                       colorscale=px.colors.diverging.RdBu,
#                       zmin=-1, zmax=1)
corr_map = go.Heatmap(x=df_corr.columns, y=df_corr.columns, z=df_corr,
                     colorscale=px.colors.diverging.RdBu)

In [ ]:
corr_title = 'Correlation Matrix'

layout = go.Layout(
    title_text=corr_title,  
    width=900, 
    height=900,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    # yaxis_autorange='reversed'
)
corr_fig=go.Figure(data=[corr_map], layout=layout)

In [ ]:
fig=go.Figure(data=[corr_map], layout=layout)

In [ ]:
fig.show()

In [ ]:
import plotly.io as pio